### Import libraries yang dibutuhkan

In [34]:
import numpy as np
import tensorflow as tf

### Load Data testing untuk semua skenario

In [35]:
x_test=np.load('./x_test.npy')
x_test,x_test.shape

(array([[ 216, 1811,  308, ...,   41,   50,  157],
        [ 399,  345,   56, ...,    0,    0,    0],
        [ 467,  360,  116, ...,    0,    0,    0],
        ...,
        [4960,  302,  148, ...,    2,   15,  255],
        [   2,    3,   43, ...,    0,    0,    0],
        [5882,  334,   15, ...,    0,    0,    0]]),
 (521, 15))

In [36]:
y_test=np.load('./y_test.npy')
y_test,y_test.shape

(array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.]], dtype=float32),
 (521, 3))

### Definisikan metrics f1-score

In [29]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Mencoba Skenario Sederhana Arsitektur

In [30]:
lstm_model=tf.keras.models.load_model('../Model/Simple/Architecture/lstm.h5',custom_objects={'f1_m':f1_m})
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15, 16)            111920    
                                                                 
 lstm_2 (LSTM)               (None, 15, 64)            20736     
                                                                 
 flatten_2 (Flatten)         (None, 960)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 2883      
                                                                 
Total params: 135,539
Trainable params: 135,539
Non-trainable params: 0
_________________________________________________________________


In [31]:
lstm_model.evaluate(x_test,y_test)

17/17 [==============================] - 1s 6ms/step - loss: 0.9279 - accuracy: 0.7831 - f1_m: 0.7868


[0.9279114603996277, 0.7831094264984131, 0.786816418170929]

In [32]:
bilstm_model=tf.keras.models.load_model('../Model/Simple/Architecture/bilstm.h5',custom_objects={'f1_m':f1_m})
bilstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 16)            111920    
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          41472     
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [33]:
bilstm_model.evaluate(x_test,y_test)

17/17 [==============================] - 1s 7ms/step - loss: 0.7966 - accuracy: 0.8157 - f1_m: 0.8197


[0.7966405749320984, 0.8157389760017395, 0.8197032809257507]

### Mencoba skenario sederhana activation function

In [9]:
import os
BASE_PATH='../Model/Simple/Activation/'

In [10]:
for model in os.listdir(BASE_PATH):
    template=f'Model {model[:-3]}'
    model=tf.keras.models.load_model(os.path.join(BASE_PATH,model),custom_objects={'f1_m':f1_m})
    print('Evaluating',template)
    model.evaluate(x_test,y_test)

Evaluating Model relu_sigmoid
17/17 [==============================] - 1s 16ms/step - loss: 0.5836 - accuracy: 0.8273 - f1_m: 0.8319
Evaluating Model relu_softmax
17/17 [==============================] - 1s 16ms/step - loss: 0.4453 - accuracy: 0.8426 - f1_m: 0.8420
Evaluating Model sigmoid_sigmoid
17/17 [==============================] - 1s 12ms/step - loss: 0.9002 - accuracy: 0.7946 - f1_m: 0.7146
Evaluating Model sigmoid_softmax
17/17 [==============================] - 1s 13ms/step - loss: 0.8918 - accuracy: 0.7524 - f1_m: 0.7598
Evaluating Model tanh_sigmoid
17/17 [==============================] - 1s 13ms/step - loss: 0.5462 - accuracy: 0.8177 - f1_m: 0.7745
Evaluating Model tanh_softmax
17/17 [==============================] - 1s 6ms/step - loss: 0.5089 - accuracy: 0.8157 - f1_m: 0.8182


### Mencoba skenario sederhana optimizer

In [11]:
adam=tf.keras.models.load_model('../Model/Simple/Optimizer/adam.h5',custom_objects={'f1_m':f1_m})
adam.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 16)            111920    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          41472     
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [12]:
adam.evaluate(x_test,y_test)

17/17 [==============================] - 1s 14ms/step - loss: 0.6350 - accuracy: 0.8157 - f1_m: 0.8204


[0.6350417137145996, 0.8157389760017395, 0.8204213380813599]

In [13]:
rmsprop=tf.keras.models.load_model('../Model/Simple/Optimizer/rmsprop.h5',custom_objects={'f1_m':f1_m})
rmsprop.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 16)            111920    
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          41472     
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [14]:
rmsprop.evaluate(x_test,y_test)

17/17 [==============================] - 1s 16ms/step - loss: 0.7578 - accuracy: 0.8445 - f1_m: 0.8471


[0.7578050494194031, 0.8445297479629517, 0.8470896482467651]

### Mencoba skenario kompleks arsitektur

In [15]:
lstm_complex=tf.keras.models.load_model('../Model/Complex/Architecture/lstm.h5',custom_objects={'f1_m':f1_m})
lstm_complex.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 128)           895360    
                                                                 
 lstm_12 (LSTM)              (None, 15, 128)           131584    
                                                                 
 lstm_13 (LSTM)              (None, 15, 128)           131584    
                                                                 
 lstm_14 (LSTM)              (None, 15, 128)           131584    
                                                                 
 flatten_4 (Flatten)         (None, 1920)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               983552    
                                                                 
 dropout_4 (Dropout)         (None, 512)              

In [16]:
lstm_complex.evaluate(x_test,y_test)

17/17 [==============================] - 1s 22ms/step - loss: 0.7413 - accuracy: 0.8695 - f1_m: 0.8670


[0.7412736415863037, 0.8694817423820496, 0.8670119643211365]

In [17]:
bilstm_complex=tf.keras.models.load_model('../Model/Complex/Architecture/bilstm.h5',custom_objects={'f1_m':f1_m})
bilstm_complex.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_6 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 3840)              0         
                                                      

In [18]:
bilstm_complex.evaluate(x_test,y_test)

17/17 [==============================] - 1s 32ms/step - loss: 0.3686 - accuracy: 0.8944 - f1_m: 0.8916


[0.3685898184776306, 0.8944337964057922, 0.8916236758232117]

### Mencoba skenario kompleks optimizer

In [19]:
adam_complex=tf.keras.models.load_model('../Model/Complex/Optimizer/adam.h5',custom_objects={'f1_m':f1_m})
adam_complex.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_5 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 3840)              0         
                                                      

In [20]:
adam_complex.evaluate(x_test,y_test)

17/17 [==============================] - 2s 31ms/step - loss: 0.6145 - accuracy: 0.8964 - f1_m: 0.8990


[0.614516019821167, 0.8963531851768494, 0.8990137577056885]

In [22]:
rmsprop_complex=tf.keras.models.load_model('../Model/Complex/Optimizer/rmsprop.h5',custom_objects={'f1_m':f1_m})
rmsprop_complex.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_8 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 15, 256)          394240    
 onal)                                                           
                                                                 
 flatten_4 (Flatten)         (None, 3840)              0         
                                                      

In [23]:
rmsprop_complex.evaluate(x_test,y_test)

17/17 [==============================] - 1s 30ms/step - loss: 0.3635 - accuracy: 0.8848 - f1_m: 0.8888


[0.3634621798992157, 0.8848368525505066, 0.8887892365455627]

### Detail LSTM

In [42]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Input data
x_t = np.array([[0.1, 0.2]])

# Cell state and hidden state at previous timestep
C_t_1 = np.array([[0.4]]) # previous cell state
h_t_1 = np.array([[0.5]]) # previous hidden state

# Define weights of LSTM
W_f = np.array([[0.1], [0.2], [0.3]])
W_i = np.array([[0.4], [0.5], [0.6]])
W_o = np.array([[0.7], [0.8], [0.9]])
W_c = np.array([[0.2], [0.3], [0.4]])

# Define bias of LSTM
b_f = np.array([0.1])
b_i = np.array([0.2])
b_o = np.array([0.3])
b_c = np.array([0.4])

### Forget gate

In [43]:
# Calculate forget Gate
f_t = sigmoid(np.dot(W_f.T, np.concatenate([h_t_1, x_t.T])) + b_f)
f_t

array([[0.55724785]])

### Input Gate

In [44]:
i_t = sigmoid(np.dot(W_i.T, np.concatenate([h_t_1, x_t.T])) + b_i)
i_t

array([[0.63876318]])

### Update Cell State

In [45]:
g_t = np.tanh(np.dot(W_c.T, np.concatenate([h_t_1, x_t.T])) + b_c)
C_t = f_t * C_t_1 + i_t * g_t
C_t

array([[0.5704675]])

### Output Gate

In [46]:
o_t = sigmoid(np.dot(W_o.T, np.concatenate([h_t_1, x_t.T])) + b_o)
np.dot(W_o.T, np.concatenate([h_t_1, x_t.T]))+b_o

array([[0.91]])

In [47]:
h_t = o_t * np.tanh(C_t)
h_t

array([[0.36769598]])

### Contoh Input Output Tiap Layer dari model

In [49]:
# Lihat semua layer yang ada di dalam model
adam_complex.layers

### Ambil sample data

In [71]:
sample=x_test[0]
sample

array([ 216, 1811,  308,  117,    2,    3,  505,  853, 1159, 1407,   50,
         49,   41,   50,  157])

### Embedding Layer

In [72]:
from tensorflow import keras
import numpy as np
embedding_input=adam_complex.layers[0].input
embedding_output=adam_complex.layers[0].output
embedding=keras.Model(inputs=embedding_input,outputs=embedding_output)
output_embedding=embedding.predict(np.expand_dims(sample,axis=0))
output_embedding,output_embedding.shape

(array([[[-3.3390496e-02, -3.5405055e-02,  1.2446079e-02, ...,
          -2.9801527e-02, -9.3560424e-03,  1.3990454e-02],
         [ 6.6025956e-03,  3.9240628e-02, -1.3936357e-02, ...,
           1.5326714e-02, -3.1085212e-02, -2.8663967e-03],
         [-1.9898659e-02, -2.3139851e-02,  1.1539438e-02, ...,
           1.2363722e-02,  1.5382340e-02, -2.9903125e-02],
         ...,
         [ 1.1481147e-02, -5.0291181e-02,  2.3783583e-02, ...,
          -1.0385682e-02, -3.5576275e-05, -3.9721217e-02],
         [ 1.6171680e-01,  6.1810087e-02,  7.5456724e-02, ...,
          -2.1985477e-02,  6.3160531e-02, -4.8857342e-02],
         [-2.8885260e-02, -5.0886877e-02, -1.4927165e-02, ...,
          -3.3463310e-02, -6.3958885e-03, -3.0276214e-03]]], dtype=float32),
 (1, 15, 128))

### Bi-LSTM Layers

In [73]:
bilstm_input=adam_complex.layers[1].input
bilstm_output=adam_complex.layers[1].output
bilstm=keras.Model(inputs=bilstm_input,outputs=bilstm_output)
output_bilstm=bilstm.predict(output_embedding)
output_bilstm,output_bilstm.shape

(array([[[2.2786841e-02, 5.7218885e-03, 0.0000000e+00, ...,
          5.7975460e-05, 3.6750458e-05, 3.8812381e-08],
         [1.6396634e-02, 8.1613054e-03, 2.4032993e-03, ...,
          8.2193277e-05, 5.5155739e-05, 5.3252233e-08],
         [1.4779088e-02, 1.8880121e-02, 1.7055911e-03, ...,
          1.1708638e-04, 7.4484247e-05, 7.4638713e-08],
         ...,
         [4.6308907e-03, 5.9852190e-03, 7.1798917e-03, ...,
          5.3049074e-03, 2.1899377e-03, 3.9493430e-06],
         [3.3358971e-03, 4.1845017e-03, 4.6774778e-03, ...,
          8.5320110e-03, 3.3786811e-03, 6.2597933e-06],
         [2.2060713e-03, 2.9526211e-03, 3.5509216e-03, ...,
          7.0627541e-03, 0.0000000e+00, 9.1494985e-06]]], dtype=float32),
 (1, 15, 256))

In [74]:
bilstm_input=adam_complex.layers[2].input
bilstm_output=adam_complex.layers[2].output
bilstm=keras.Model(inputs=bilstm_input,outputs=bilstm_output)
output_bilstm=bilstm.predict(output_bilstm)
output_bilstm,output_bilstm.shape

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32),
 (1, 15, 256))

In [75]:
bilstm_input=adam_complex.layers[3].input
bilstm_output=adam_complex.layers[3].output
bilstm=keras.Model(inputs=bilstm_input,outputs=bilstm_output)
output_bilstm=bilstm.predict(output_bilstm)
output_bilstm,output_bilstm.shape

(array([[[0.        , 0.0107895 , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.02093669, 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.03106452, 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.14055702, 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.14826922, 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.15538645, 0.        , ..., 0.        ,
          0.        , 0.        ]]], dtype=float32),
 (1, 15, 256))

### Flatten Layer

In [77]:
flatten_input=adam_complex.layers[4].input
flatten_output=adam_complex.layers[4].output
flatten=keras.Model(inputs=flatten_input,outputs=flatten_output)
output_flatten=flatten.predict(output_bilstm)
output_flatten,output_flatten.shape

(array([[0.       , 0.0107895, 0.       , ..., 0.       , 0.       ,
         0.       ]], dtype=float32),
 (1, 3840))

### Dense Layer

In [78]:
dense_input=adam_complex.layers[5].input
dense_output=adam_complex.layers[5].output
dense=keras.Model(inputs=dense_input,outputs=dense_output)
output_dense=dense.predict(output_flatten)
output_dense,output_dense.shape

(array([[0.        , 0.        , 1.5929614 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 1.9359404 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.81360036, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.49931765, 0.        ,
         0.        , 1.7630045 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 1.4364382 , 0.        ,
         0.        , 0.        , 1.1873258 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 3.1262105 ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.07437122, 0.        ,
         0.        , 0.        , 0

### Dropout Layer

In [68]:
dropout_input=adam_complex.layers[6].input
dropout_output=adam_complex.layers[6].output
dropout=keras.Model(inputs=dropout_input,outputs=dropout_output)
output_dropout=dropout.predict(output_dense)
output_dropout

array([[0.        , 0.        , 1.5929614 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.9359404 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.81360036, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.49931765, 0.        ,
        0.        , 1.7630045 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.4364382 , 0.        ,
        0.        , 0.        , 1.1873258 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 3.1262105 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.07437122, 0.        ,
        0.        , 0.        , 0.        , 0.  

### Dense Output Layer

In [82]:
dense_input2=adam_complex.layers[7].input
dense_output2=adam_complex.layers[7].output
dense2=keras.Model(inputs=dense_input2,outputs=dense_output2)
output_dense2=dense2.predict(output_dropout)
output_dense2,output_dense2.shape

(array([[1.1374094e-05, 7.4108061e-06, 9.9998116e-01]], dtype=float32), (1, 3))

In [91]:
np.argmax(output_dense2,axis=1),np.argmax(y_test[0],axis=0)

(array([2], dtype=int64), 2)